# `Aplicação Original`: Comparação do Cadastro de Clientes com Bases Públicas (OFAC, ONU, OFAC-NON)

## Instalando os Módulos

In [ ]:
!pip install python-Levenshtein thefuzz pyspark

## Bibliotecas Necessárias

In [2]:
    import time
    import pandas as pd
    from thefuzz import fuzz
    from pyspark.sql.functions import lit
    from pyspark.sql.functions import udf
    from pyspark.sql.functions import trim
    from pyspark.sql.functions import col
    from pyspark.sql.types import IntegerType
    from pyspark.sql import functions as F
    from pyspark.sql import SparkSession
    from pyspark.conf import SparkConf

## Inicialização da Sessão do Spark

In [3]:
import warnings
warnings.filterwarnings("ignore")

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL - Aplicação Original") \
    .getOrCreate()

## Path das Bases

In [4]:
file_renov = "nomes_clientes_banco.csv"
file_lista_risco = "base_ofac.csv"

## Base Renovação

In [ ]:
df_renov = spark.read \
    .format("csv") \
    .option("delimiter", ",") \
    .option("header", True) \
    .option("ignoreLeadingWhiteSpace", True) \
    .option("ignoreTrailingWhiteSpace", True) \
    .load(file_renov)

df_renov = df_renov.withColumn("NOME",trim(col("NOME")))

df_renov = df_renov.withColumn("m", lit(1))

df_renov.show()

## Base Lista de Risco

In [ ]:
df_lista_risco = spark.read \
    .format("csv") \
    .option("delimiter", ",") \
    .option("header", True) \
    .option("encoding", "cp1252") \
    .option("ignoreLeadingWhiteSpace", True) \
    .option("ignoreTrailingWhiteSpace", True) \
    .load(file_lista_risco)

df_lista_risco = df_lista_risco.dropDuplicates()

df_lista_risco = df_lista_risco.withColumnRenamed("NOME","NOME_RISCO")

df_lista_risco = df_lista_risco.withColumn("m", lit(1))

df_lista_risco.show()

## Operação de Inner Join entre as Bases

In [ ]:
df_match = df_renov.join(df_lista_risco, ((df_renov['TpPessoa']==df_lista_risco['TIPO']) & (df_renov['m']==df_lista_risco['m'])), how='inner')

df_match = df_match.drop('m')

df_match.show()

print((df_match.count(), len(df_match.columns)))

## Cálculo de Similaridade entre as Bases

In [ ]:
# Apply Levenshtein
# udf function
def lev(x, y):
    return fuzz.ratio(x,y)

similarity = udf(lev, IntegerType())

# using udf
df_match = df_match.withColumn("SIMILARITY", similarity(df_match.NOME,df_match.NOME_RISCO))

filtered_df = df_match.filter(df_match.SIMILARITY > 90)

df_match.show()

## Lista de Sanções

In [ ]:
filtered_df.show()